# expID: Description

## Preparations

### Import required libraries

In [ ]:
library(tidyverse)

### Define variables 

In [ ]:
expID <- 'ABC123'
dish_volume = 2 # Volume of cell suspension from which sample for the CASY measurement is taken
seeding_density <- 0.8e5

Expects the following folder structure:

+-- raw_data
| +-- expID_cell_concentration.csv # Cell concentration taken from CASY\
| +-- expID_volume.csv # Cell volume taken from CASY\
| +-- expID_timetable.csv # File containing start and end times of each time point\

See example files for internal structure

### Load data

In [ ]:
cc <- read.csv(paste('raw_data/', expID, '_cell_concentration.csv', sep=''))
v <- read.csv(paste('raw_data/', expID, '_volume.csv', sep=''))
tt <- read.csv(paste('raw_data/', expID, '_timetable.csv', sep=''))

### Convert datetime to correct format

In [ ]:
tt <- tt %>%
    mutate(t_end = strptime(t_end, "%Y-%m-%d %H:%M"), # Convert t_end to dttm
           t_start = strptime(t_start, "%Y-%m-%d %H:%M"), # Convert t_start to dttm
           )

### Create vector with time points

In [ ]:
timepoints <- levels(tt$timepoint)

## Raw data

### Cell concentration

In [ ]:
cc

### Mean volume

In [ ]:
v

### Timetable

In [ ]:
tt

## Data manipulation

### Denormalization

In [ ]:
cc_denorm <- cc %>%
    pivot_longer(all_of(timepoints), names_to = 'timepoint', values_to = 'cell_concentration')

v_denorm <- v %>%
    pivot_longer(all_of(timepoints), names_to = 'timepoint', values_to = 'volume')

### Merge

In [ ]:
data <- merge(x = cc_denorm, y = v_denorm, by = c('sample_id','medium','replicate','timepoint'), all = TRUE)
data <- merge(x = data, y = tt, by = c('medium','timepoint'), all.x = TRUE)

### Calculate hours

In [ ]:
data <- data %>%
    group_by(sample_id) %>%
    mutate(hours = as.double(difftime(t_end, lag(t_start), units='hours')), # Calculate time difference in hours
           cumulative_hours = cumsum(replace_na(hours, 0)) # Calculate cumulative hours
          )

### Calculate cell number

Cell concentration is given in cells/1mL.\
Total volume was 2mL, therefore cell concentration has to be multiplied with 2.

In [ ]:
data <- data %>%
    mutate(cell_number = case_when(timepoint != 't0' ~ cell_concentration*dish_volume, # When timepoint is NOT t0 multiply with volume
                                   timepoint == 't0' ~ seeding_density # When timepoint is t0, set to seeding_density
                                  )
          )

### Calculate cumulative cell number

Cumulative cell number (ccn) is calculated in the following way, with 1e5cells as the seeding density as an example:

fold_change(t) = cell_number(t)/1e5 (Equation 1)\
ccn(t) = ccn(t-1) * fold_change(t) (Equation 2)

Equation 2 is equivalent to:\
ccn(t) = cumprod(fold_change) * 1e5 (Equation 3)\
and implemented in this form.

1e5 is the initial number of seeded cells.

In [ ]:
data <- mutate(data, fold_change = cell_number/seeding_density)

data <- data %>%
    group_by(sample_id) %>%
    mutate(cumulative_cell_number = cumprod(fold_change)*seeding_density) # Equation 3

### Calculate doubling time

$dt=T\frac{ln(2)}{ln(\frac{N_1}{N_0})}$

In [ ]:
data <- data %>%
    mutate(doubling_time = hours * log(2)/log(cell_number/seeding_density))

### Order dataframe

In [ ]:
# Define order of levels
#data$medium <- factor(data$medium, levels = c('medium1','medium2','medium3','medium4'))

# Order dataframe by specified columns
data <- data %>%
    arrange(sample_id, timepoint)


# Order sequence of columns
data <- data[,c('sample_id',
                'medium',
                'replicate',
                'timepoint',
                'cell_concentration',
                'cell_number',
                'fold_change',
                'cumulative_cell_number',
                'doubling_time',
                'hours',
                'cumulative_hours',
                't_end',
                't_start',
                'volume'
               )]

### Data

In [ ]:
data

## Export as csv

In [ ]:
write.csv(data, paste(expID, '_data.csv', sep=""), row.names=FALSE)